# Modelo LLaMA Fine-tuned

Este entrenamiento está basado en el modelo [LLaMA 2 7B](https://huggingface.co/NousResearch/Llama-2-7b-hf), y los datos de entrenamiento provienen del Corpus ClearSim.

El token de Hugging Face está omitido por motivos de seguridad.

## Importar librerías y corpus de datos

In [ ]:
!pip install trl accelerate bitsandbytes transformers peft --quiet

import torch
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline, logging
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from google.colab import files
from datasets import load_dataset

# Iniciar sesión de Hugging Face
!huggingface-cli login

In [ ]:
# Importar corpus (formato JSONLines)
uploaded = files.upload()
dataset = load_dataset("json", data_files="datos.jsonl", split="train") # Cambio de formato a JSONLines

## Configuración de parámetros y modelo

In [ ]:
# Configuración cuantización
bnb_argumentos = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# Modelo
modelo = AutoModelForCausalLM.from_pretrained(
    "NousResearch/Llama-2-7b-hf",
    quantization_config=bnb_argumentos,
    device_map={"": 0}
)

# Tokenizador
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-hf", trust_remote_code=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Configuración de LoRA
peft_argumentos = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    task_type="CAUSAL_LM",
    bias="none",
)

# Parámetros de entrenamiento
train_argumentos = TrainingArguments(
    output_dir="./resultados",
    num_train_epochs=5,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-3,
    weight_decay=0.001,
    bf16=True,
    max_grad_norm=0.4,
    max_steps=-1,
    warmup_ratio=0.04,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)
trainer = SFTTrainer(
    model=modelo,
    train_dataset=dataset,
    peft_config=peft_argumentos,
    args=train_argumentos
)

## Entrenamiento

In [ ]:
trainer.train()
trainer.model.save_pretrained("e2r-finetune")

## Subir a nuevo repositorio de Hugging Face

In [ ]:
from huggingface_hub import login, create_repo

login("HF-TOKEN") # Aquí iría el token
repo_name = "e2r-finetuned"
create_repo(repo_id=repo_name, private=False)
trainer.model.push_to_hub("Nizaress/e2r-finetuned")

# Uso del modelo

Aquí tenemos un ejemplo de cómo utilizar el modelo para generar texto.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig

def cargar_modelo(repo_adaptador: str, hf_token: str | None = None):
    config = PeftConfig.from_pretrained(repo_adaptador, token=hf_token)
    modelo_base = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, token=hf_token)
    modelo_peft = PeftModel.from_pretrained(modelo_base, repo_adaptador, token=hf_token)

    tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, token=hf_token)
    return modelo_peft.eval(), tokenizer

texto_original = "Prueba del modelo."

model, tokenizer = cargar_modelo("Nizaress/e2r-finetuned", hf_token="HF-TOKEN") # En hf_token colocas tu token

inputs = tokenizer(texto_original, return_tensors="pt").to(model.device)

output = model.generate(
    **inputs,
    max_new_tokens=500,
    do_sample=True,
    temperature=0.5
)

generated = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated[len(texto_original):])